## 専門知識労働者
- 保険テクノロジー企業である Insurellm の従業員が使用する、専門知識を持つナレッジ ワーカーである質問応答エージェント。エージェントは正確である必要があり、ソリューションは低コストである必要があります。
- このプロジェクトでは、RAG（検索拡張生成）を使用して、質問/回答アシスタントが高い精度を確保します。ココ、2つめの実装では、RAGにおけるチャンキングを試行します。

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [ ]:
# requirements.txt を確認して langchain系をインストール
!pip install langchain
!pip install langchain-core
!pip install langchain-text-splitters
!pip install langchain-openai
!pip install langchain-chroma
!pip install langchain-community

In [3]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [4]:
# 価格は重要な要素（と言う建付け）なので、低コストモデルを採用

MODEL = "gpt-4o-mini"
# db_name = "vector_db" # 使ってない

In [5]:
# .envファイルから環境変数をロード
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [6]:
# langChain のローダーを使用してKBのすべてのサブフォルダ内のすべてのドキュメントを読み取りリスト化（ここでバグを修正してくれた Mark D. と Zoya H. に感謝します。
# 余談：メタデータは、DirectoryLoader および TextLoader によって生成され、そこにカスタムの属性、doc_typeを追加している。

folders = glob.glob("knowledge-base/*")

# 一部のユーザーに必要な修正を提供してくれた、コース受講生のCGとJon Rに感謝します。
text_loader_kwargs = {'encoding': 'utf-8'}
# それでもうまくいかない場合は、Windowsユーザーの中には次の行のコメントを解除する必要があるかもしれません。
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    # doc は LangChain の Document クラスで、metadata フィールドを持つ
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [7]:
# ドキュメント・リストの長さ
len(documents)

31

In [8]:
# 24番のドキュメント
documents[24]

Document(metadata={'source': 'knowledge-base/employees/Oliver Spencer.md', 'doc_type': 'employees'}, page_content='# HR Record\n\n# Oliver Spencer\n\n## Summary\n- **Date of Birth**: May 14, 1990  \n- **Job Title**: Backend Software Engineer  \n- **Location**: Austin, Texas  \n\n## Insurellm Career Progression\n- **March 2018**: Joined Insurellm as a Backend Developer I, focusing on API development for customer management systems.\n- **July 2019**: Promoted to Backend Developer II after successfully leading a team project to revamp the claims processing system, reducing response time by 30%.\n- **June 2021**: Transitioned to Backend Software Engineer with a broader role in architecture and system design, collaborating closely with the DevOps team.\n- **September 2022**: Assigned as the lead engineer for the new "Innovate" initiative, aimed at integrating AI-driven solutions into existing products.\n- **January 2023**: Awarded a mentorship role to guide new hires in backend technology a

In [9]:
# テキストを200文字の重複部分を持たせた1000文字ごとのチャンク（かたまり）に分割
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# document も chunk も LangChain の Document クラス
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [10]:
# チャンク・リストの長さ
len(chunks)

123

In [11]:
# 6番のチャンク
chunks[6]

Document(metadata={'source': 'knowledge-base/contracts/Contract with Apex Reinsurance for Rellm.md', 'doc_type': 'contracts'}, page_content="1. **Technical Support**: Provider shall offer dedicated technical support to the Client via phone, email, and a ticketing system during business hours (Monday to Friday, 9 AM to 5 PM EST).\n\n2. **Training and Onboarding**: Provider will deliver comprehensive onboarding training for up to ten (10) members of the Client's staff to ensure effective use of the Rellm solution.\n\n3. **Updates and Maintenance**: Provider is responsible for providing updates to the Rellm platform to improve functionality and security, at no additional cost to the Client.\n\n4. **Escalation Protocol**: Issues that cannot be resolved at the first level of support will be escalated to the senior support team, ensuring that critical concerns are addressed promptly.\n\n---\n\n**Acceptance of Terms**: By signing below, both parties agree to the Terms, Renewal, Features, and 

In [12]:
# chunks に含まれるすべての要素から doc_type メタデータを抽出し、重複を除いた種類を一覧表示
# ココでは、従業員、契約、製品、会社などの属性（元はフォルダ名）が表示されるハズ
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: employees, contracts, products, company


In [13]:
# CEOと言う文字列を含む（３つの）チャンクを出力している。
# chunk は LangChain の Document クラスで、page_content フィールドを持つ
for chunk in chunks:
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='3. **Regular Updates:** Insurellm will offer ongoing updates and enhancements to the Homellm platform, including new features and security improvements.

4. **Feedback Implementation:** Insurellm will actively solicit feedback from GreenValley Insurance to ensure Homellm continues to meet their evolving needs.

---

**Signatures:**

_________________________________  
**[Name]**  
**Title**: CEO  
**Insurellm, Inc.**

_________________________________  
**[Name]**  
**Title**: COO  
**GreenValley Insurance, LLC**  

---

This agreement represents the complete understanding of both parties regarding the use of the Homellm product and supersedes any prior agreements or communications.' metadata={'source': 'knowledge-base/contracts/Contract with GreenValley Insurance for Homellm.md', 'doc_type': 'contracts'}
_________
page_content='## Support

1. **Customer Support**: Velocity Auto Solutions will have access to Insurellm’s customer support team via email or chatbot, availabl